### Notebook: jacobian_beta_to_k.ipynb

**Objective:** This notebook demonstrates how the intrinsic QH decay rate `β/α`, defined in proper time, maps to the observed exponential decay constant `k` in redshift space. We will show that the cosmological time-dilation effect, known as the Jacobian, naturally accounts for the factor-of-10 difference between the lab value of `β` and the cosmologically observed `k`.


#### 1. Setup and Constants

First, we import the necessary libraries and define the core QH and cosmological parameters. We'll use standard Planck 2018 values for the cosmology.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u

# QH Parameters
alpha = 0.314
beta_lab = 0.0158

# Observational Results from MCMC fit
k_obs = 0.521
k_err = 0.057

# Cosmological Parameters (Planck 2018)
H0 = 67.4
Om0 = 0.315

# Define the cosmology using Astropy
cosmo = FlatLambdaCDM(H0=H0, Om0=Om0)


#### 2. The Time-Redshift Jacobian

The effective scaling factor between the intrinsic decay rate (`β/α`) and the observed one (`k`) is proportional to the function $C(z) = (1+z)H(z)/H_0$. This factor quantifies how much "faster" the universe evolves in redshift space compared to proper time at a given epoch. A decay process with a constant rate in time will appear to have a redshift-dependent rate `k_pred(z)`.

$$k_{\\rm pred}(z) = \\frac{\\beta_{\\rm lab}}{\\alpha} \\times \\frac{(1+z)H(z)}{H_0}$$

We can now calculate this predicted `k` across our redshift range of interest, `z = [4, 8]`.


In [ ]:
# Create a redshift range for plotting
z_range = np.linspace(0, 10, 200)

# Calculate the Jacobian scaling factor C(z)
C_z = (1 + z_range) * cosmo.H(z_range).value / cosmo.H0.value

# Calculate the predicted k(z)
k_predicted_z = (beta_lab / alpha) * C_z

# Find the average predicted k in the MIDIS survey range [4, 8]
midis_mask = (z_range >= 4) & (z_range <= 8)
k_predicted_avg = np.mean(k_predicted_z[midis_mask])

print(f"Intrinsic decay rate (β/α): {beta_lab/alpha:.4f}")
print(f"Average Jacobian factor C(z) for z in [4, 8]: {np.mean(C_z[midis_mask]):.2f}")
print(f"Predicted k from Jacobian mapping: {k_predicted_avg:.3f}")


#### 3. Visualization and Comparison

Now we plot the predicted `k(z)` and compare its average value in the JWST/MIDIS redshift range to the value of `k=0.521` we observed in our MCMC fit.


In [ ]:
plt.style.use('seaborn-v0_8-whitegrid')
fig, ax = plt.subplots(figsize=(10, 6))

# Plot the predicted k(z)
ax.plot(z_range, k_predicted_z, color='darkslateblue', lw=2, label=r'Predicted $k(z) = (\beta/\alpha) \times C(z)$')

# Highlight the observed k value and its uncertainty
ax.axhspan(k_obs - k_err, k_obs + k_err, color='crimson', alpha=0.2, label=f'Observed $k = {k_obs} \pm {k_err}$')
ax.axhline(k_obs, color='crimson', ls='--', lw=1.5)

# Highlight the MIDIS redshift range
ax.axvspan(4, 8, color='gray', alpha=0.15, label='MIDIS z Range')

# Mark the average predicted value in that range
ax.plot([4, 8], [k_predicted_avg, k_predicted_avg], color='dodgerblue', ls='-', marker='|', ms=10, lw=2, label=f'Avg. Predicted $k = {k_predicted_avg:.3f}$')

ax.set_xlabel('Redshift z', fontsize=14)
ax.set_ylabel('Effective Decay Constant k', fontsize=14)
ax.set_title('QH Jacobian Mapping: Intrinsic β to Observed k', fontsize=16)
ax.legend(fontsize=12)
ax.set_xlim(0, 10)
ax.set_ylim(0, 1.0)
ax.tick_params(axis='both', which='major', labelsize=12)

# Save the figure
plt.savefig('../figures/jacobian_factor.png', dpi=150, bbox_inches='tight')
plt.show()
